In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Get:12 http://archive.ubuntu.com/ubun

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
apparel_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
apparel_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = apparel_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
vine_votes = vine_df.filter("total_votes>=20")
vine_votes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
vine_filtered = vine_votes.filter("helpful_votes/total_votes>=0.5")
vine_filtered.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
vine_reviews_yes = vine_filtered.filter("vine == 'Y'")
vine_reviews_yes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [17]:
vine_yes_count = vine_reviews_yes.count()
vine_yes_count

33

In [9]:
vine_reviews_no = vine_filtered.filter("vine == 'N'")
vine_reviews_no.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [18]:
vine_no_count = vine_reviews_no.count()
vine_no_count

45388

In [10]:
total_reviews = vine_df.count()
total_reviews

5906333

In [13]:
five_star_count = vine_df.filter("star_rating == 5").count()
five_star_count

3320557

In [14]:
vine_paid_fivestar = vine_reviews_yes.filter("star_rating==5").count()
vine_paid_fivestar

15

In [19]:
paid_percent = (vine_paid_fivestar / vine_yes_count) * 100
paid_percent

45.45454545454545

In [20]:
vine_unpaid_fivestar = vine_reviews_no.filter("star_rating==5").count()
vine_unpaid_fivestar

23733

In [21]:
unpaid_percent = (vine_unpaid_fivestar / vine_no_count) * 100
unpaid_percent

52.28915131752886